In [14]:
# Libraries

import keras

Using TensorFlow backend.


Load data stored online

In [0]:
# Data loading method

def load_data_from_url(url, txt_file_name="text_file.txt"):
  path_to_file = keras.utils.get_file(txt_file_name, url)
  # Read, then decode for py2 compat.
  text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
  return text

Perform pre-processing operatiorns

In [0]:
# Processing method

def process_text(text, min_word_frequency=10, seq_len=10):
  '''Performs preprocessing step for a given text corpus'''
  clean_text = text.replace('\n', ' \n ')
  # Split with new lines consideration
  clean_text = [w for w in clean_text.split(' ') if w.strip() != '' or w == '\n']

  # Work upon word freqs.
  word_freq = {}
  for word in clean_text:
    word_freq[word] = word_freq.get(word, 0) + 1
    
  # Check for min frequency condition

  ignored_words = set()
  for k, v in word_freq.items():
    if word_freq[k] < min_word_frequency:
      ignored_words.add(k)

  words = set(clean_text)
  print('Unique words before ignoring:', len(words))
  print('Ignoring words with frequency <', min_word_frequency)
  words = sorted(set(words) - ignored_words)
  print('Unique words after ignoring:', len(words))
  # cut the text in semi-redundant sequences of SEQUENCE_LEN words
  STEP = 1
  sentences = []
  next_words = []
  ignored = 0
  for i in range(0, len(clean_text) - seq_len, STEP):
      # Only add sequences where no word is in ignored_words
      if len(set(clean_text[i: i+seq_len+1]).intersection(ignored_words)) == 0:
          sentences.append(clean_text[i: i + seq_len])
          next_words.append(clean_text[i + seq_len])
      else:
          ignored = ignored+1
  print('Ignored sequences:', ignored)
  print('Remaining sequences:', len(sentences))

  return (word_freq, words, ignored_words, sentences, next_words)

In [0]:
def shuffle_and_split_training_set(sentences_original, next_original, percentage_test=2):
    '''Performs split of dataset'''
    # shuffle at unison
    print('Shuffling sentences')

    tmp_sentences = []
    tmp_next_word = []
    for i in np.random.permutation(len(sentences_original)):
        tmp_sentences.append(sentences_original[i])
        tmp_next_word.append(next_original[i])

    cut_index = int(len(sentences_original) * (1.-(percentage_test/100.)))
    x_train, x_test = tmp_sentences[:cut_index], tmp_sentences[cut_index:]
    y_train, y_test = tmp_next_word[:cut_index], tmp_next_word[cut_index:]

    print("Size of training set = %d" % len(x_train))
    print("Size of test set = %d" % len(y_test))
    return (x_train, y_train), (x_test, y_test)

In [0]:
data_url = 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'

In [0]:
result = load_data_from_url(data_url)

In [22]:
(word_freq, words, ignored_words, sentences, next_words) = process_text(result)

Unique words before ignoring: 25671
Ignoring words with frequency < 10
Unique words after ignoring: 2154
Ignored sequences: 219742
Remaining sequences: 22899
